<a href="https://colab.research.google.com/github/raja-jamwal/blog-building-agentic-architectures/blob/main/langchain/Part_6_Resource_Awareness%2C_Reasoning%2C_and_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

We have built agents that are smart, collaborative, and connected. Now, we need to make them **efficient** and **safe**.

In production, you cannot afford to run every "Hello World" query through the most expensive model (GPT-4o/Claude 3.5 Sonnet). Nor can you afford to let an agent output unstructured garbage that breaks your frontend.

We will cover:
1.  **Resource-Aware Optimization:** Dynamically routing tasks to the cheapest model that can handle them.
2.  **Reasoning Techniques:** Forcing the model to "think" (Chain of Thought) to solve hard problems.
3.  **Guardrails:** Enforcing strict output structures and safety checks.

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o vs GPT-4o-mini)
*   **Pydantic** (For data validation)

### The n8n Connection
*   **Resource-Awareness** = A **Switch Node** that checks the complexity of the input. Path A goes to a "Basic LLM" node (Mini), Path B goes to an "Advanced LLM" node.
*   **Reasoning** = Prompt Engineering within the AI Agent node. Adding "Let's think step by step" to the system prompt.
*   **Guardrails** = The **Structured Output Parser** in n8n (often found in the Advanced AI nodes) or using an **If Node** to check if the output contains forbidden words.

In [1]:
# @title 1. Install Dependencies
!pip install -qU langchain langchain-openai langchain-core pydantic

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize Models
from langchain_openai import ChatOpenAI

# We need two distinct models for the optimization pattern
cheap_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
expensive_llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✅ Environment Setup Complete.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 16: Resource-Aware Optimization

**The Problem:** Using a flagship model (like GPT-4o) for simple tasks (like "What is the capital of France?") is a waste of money and adds latency.

**The Solution:** **Dynamic Routing**. We use a tiny, fast model to classify the *complexity* of the request. Based on that classification, we route the work to the appropriate model.

**The Scenario:**
*   **Simple:** "What is 2+2?" -> Route to `gpt-4o-mini`.
*   **Complex:** "Write a Python script to scrape a website and analyze sentiment." -> Route to `gpt-4o`.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

# --- 1. The Classifier (The Traffic Cop) ---
# We use the cheap model to decide where to send the request.
classifier_prompt = ChatPromptTemplate.from_template(
    """Given the user request below, classify it as either 'SIMPLE' or 'COMPLEX'.
    - Simple: Factual questions, greetings, basic summaries.
    - Complex: Coding, creative writing, reasoning, multi-step logic.

    Return ONLY the word 'SIMPLE' or 'COMPLEX'.

    Request: {question}"""
)
classifier_chain = classifier_prompt | cheap_llm | StrOutputParser()

# --- 2. The Branches ---
chain_simple = (
    ChatPromptTemplate.from_template("You are a fast assistant. Answer briefly: {question}")
    | cheap_llm
    | StrOutputParser()
)

chain_complex = (
    ChatPromptTemplate.from_template("You are an expert reasoner. Provide a detailed answer: {question}")
    | expensive_llm
    | StrOutputParser()
)

# --- 3. The Routing Logic ---
branch = RunnableBranch(
    (lambda x: "SIMPLE" in x["topic"].upper(), chain_simple),
    chain_complex # Default fallback (usually the stronger model)
)

# --- 4. The Full Chain ---
full_chain = (
    {"topic": classifier_chain, "question": RunnablePassthrough()}
    | branch
)

# --- Execution ---
print("--- Test 1: Simple Query ---")
q1 = "Who is the president of the USA?"
print(f"Q: {q1}")
# This should route to Mini (Fast)
print(f"A: {full_chain.invoke(q1)}\n")

print("--- Test 2: Complex Query ---")
q2 = "Write a Python class for a Binary Search Tree with insert and delete methods."
print(f"Q: {q2}")
# This should route to GPT-4o (Slow but smart)
# Note: We won't print the whole code to save space, just the start.
result = full_chain.invoke(q2)
print(f"A: {result[:100]}...")

--- Test 1: Simple Query ---
Q: Who is the president of the USA?
A: As of October 2023, the president of the USA is Joe Biden.

--- Test 2: Complex Query ---
Q: Write a Python class for a Binary Search Tree with insert and delete methods.
A: Creating a Binary Search Tree (BST) in Python involves defining a class for the tree itself and a cl...


## Pattern 17: Reasoning Techniques (Chain of Thought)

**The Problem:** If you ask an LLM a trick question, it often guesses the answer immediately based on probability, which leads to errors.

**The Solution:** **Chain of Thought (CoT)**. We explicitly instruct the model to "think step-by-step" before answering. This forces the model to generate intermediate tokens, which act as a "scratchpad" for its logic.

**The Scenario:** A classic logic puzzle that models often get wrong without reasoning.

In [3]:
# --- The Puzzle ---
puzzle = "If I hang 5 shirts to dry and they take 5 hours to dry, how long will it take to dry 10 shirts?"

# --- Approach 1: Zero-Shot (Direct Answer) ---
# Often fails or gives a quick, wrong answer.
direct_prompt = ChatPromptTemplate.from_template("Answer this: {puzzle}")
direct_chain = direct_prompt | cheap_llm | StrOutputParser()

# --- Approach 2: Chain of Thought (Reasoning) ---
# Forces the model to break it down.
cot_prompt = ChatPromptTemplate.from_template(
    """Answer the following question.
    CRITICAL: You must think step-by-step.
    1. Identify the variables.
    2. Analyze the constraints.
    3. Formulate the logic.
    4. Give the final answer.

    Question: {puzzle}"""
)
cot_chain = cot_prompt | cheap_llm | StrOutputParser()

# --- Execution ---
print(f"🧩 Puzzle: {puzzle}\n")

print("--- ❌ Direct Answer (Risk of Error) ---")
print(direct_chain.invoke({"puzzle": puzzle}))

print("\n--- ✅ Chain of Thought (Robust) ---")
print(cot_chain.invoke({"puzzle": puzzle}))

🧩 Puzzle: If I hang 5 shirts to dry and they take 5 hours to dry, how long will it take to dry 10 shirts?

--- ❌ Direct Answer (Risk of Error) ---
If you hang 10 shirts to dry at the same time as the 5 shirts, they will still take 5 hours to dry, assuming there is enough space and airflow for all the shirts to dry simultaneously. So, it will take 5 hours to dry 10 shirts.

--- ✅ Chain of Thought (Robust) ---
1. **Identify the variables:**
   - Let \( S \) be the number of shirts.
   - Let \( T \) be the time taken to dry the shirts.
   - From the question, we know that \( S = 5 \) shirts take \( T = 5 \) hours to dry.

2. **Analyze the constraints:**
   - The drying process is assumed to be independent of the number of shirts, meaning that drying 5 shirts takes the same amount of time as drying 10 shirts, provided there is enough space to hang all the shirts simultaneously.
   - If the drying space is limited and cannot accommodate all shirts at once, the drying time would increase. Ho

## Pattern 18: Guardrails (Safety & Structure)

**The Problem:** You are building an API that expects JSON output. If the LLM adds "Here is your JSON:" before the actual JSON, your code breaks. Or worse, the LLM generates toxic content.

**The Solution:** **Pydantic Parsers**. We define a strict data class (Schema). We pass this schema to the LLM. The LLM *must* return data that fits this shape, or the system throws a validation error.

**The Scenario:** We want to extract sentiment from a review, but it MUST be strictly formatted as `{"sentiment": "POS/NEG", "score": 0-10}`.

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator

# --- 1. Define the Guardrail (Schema) ---
class SentimentAnalysis(BaseModel):
    sentiment: str = Field(description="Must be either 'POSITIVE' or 'NEGATIVE'")
    score: int = Field(description="A score between 1 and 10")
    reasoning: str = Field(description="A brief explanation")

    @field_validator("score")
    @classmethod
    def check_score_range(cls, value: int):
        if not 1 <= value <= 10:
            raise ValueError("Score must be between 1 and 10!")
        return value

# --- 2. Setup the Parser ---
parser = PydanticOutputParser(pydantic_object=SentimentAnalysis)

# --- 3. The Prompt with Instructions ---
# parser.get_format_instructions() automatically generates the prompt
# telling the LLM how to format the JSON.
prompt = ChatPromptTemplate.from_template(
    """Analyze the review below.
    {format_instructions}

    Review: {review}"""
)

# --- 4. The Chain ---
# The parser is the final step. It takes the text and converts it to a Python Object.
safe_chain = prompt | cheap_llm | parser

# --- Execution ---
review_text = "The food was okay, but the service was terrible. I waited 40 minutes."

try:
    print("--- 🛡️ Executing Guardrailed Chain ---")
    result = safe_chain.invoke({
        "review": review_text,
        "format_instructions": parser.get_format_instructions()
    })

    print(f"Type: {type(result)}") # It's a Pydantic object, not a string!
    print(f"Sentiment: {result.sentiment}")
    print(f"Score: {result.score}")
    print(f"Reasoning: {result.reasoning}")

except Exception as e:
    print(f"Guardrail Triggered! Validation Failed: {e}")

--- 🛡️ Executing Guardrailed Chain ---
Type: <class '__main__.SentimentAnalysis'>
Sentiment: NEGATIVE
Score: 4
Reasoning: The food was only okay, and the service was poor with a long wait time.


## Summary

You have now engineered a system that is:

1.  **Cost-Effective:** It uses the right model for the right task.
2.  **Smart:** It thinks before it speaks.
3.  **Safe:** It guarantees structured, valid outputs.

In **Part 7**, the final chapter, we will look at **Strategy**. We will build an **Evaluation Pipeline** (LLM-as-a-Judge) to automatically grade our agents.